In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from tqdm import tqdm

import numpy as np
import pandas as pd

In [2]:
samples_per_city = pd.read_excel('yelp data_Lee.xlsx')
samples_per_city.columns = samples_per_city.iloc[0]
samples_per_city = samples_per_city.iloc[1:].set_index('City')[['per 200']]
samples_per_city.columns = ['sample']
samples_per_city.head(3)

,sample
City,
Phoenix,5
Scottsdale,4
Tempe,5


In [3]:
city_links = pd.read_csv('city_links.csv').set_index('name')
samples_per_city = samples_per_city.join(city_links, how='inner')
samples_per_city.head(3)

,sample,url
Alameda,5,/alameda-ca-us
Albany,2,/albany-ca-us
Albuquerque,5,/albuquerque


# Get restaurants from city

In [4]:
profile = webdriver.FirefoxProfile()
profile.set_preference('permissions.default.image', 2)
profile.set_preference('dom.ipc.plugins.enabled.libflashplayer.so', 'false')
restaurants = []
for city in tqdm(samples_per_city.index):
    
    sample_size, url = samples_per_city.loc[city][['sample', 'url']]
    
    driver = webdriver.Firefox(firefox_profile=profile)
    try:
        driver.get('https://www.yelp.com' + url);
    except:
        driver.close()
        continue
    driver.find_element_by_xpath('//a[text()="%s"]' % 'Restaurants').click();
    count_text = driver.find_element_by_xpath('//span[@class="pagination-results-window"]').text
    
    n_restaurants = int(count_text.split(' ')[-1])
    n_pages = max(n_restaurants // 10, 1)
    n_pages = min(n_pages, 100)
    
    # random start
    #np.random.seed(0)
    max_reachable_resturants = min(n_restaurants, 990)
    start = np.random.randint(0, max_reachable_resturants-1)
    
    selected_restaurant_numbers = [start+1]

    base_url = driver.current_url
    for _ in range(sample_size):

        driver.get(base_url + '&start=%d&cflt=restaurants' % start)
        start = (start + 200) % max_reachable_resturants

        while start+1 in selected_restaurant_numbers:
            start += 1
        selected_restaurant_numbers.append(start)

        base_xpath = '//li[@class="regular-search-result"][1]'

        link = driver.find_element_by_xpath(base_xpath + '//a[@class="biz-name js-analytics-click"]').get_attribute('href')


        name = driver.find_element_by_xpath(base_xpath + '//a[@class="biz-name js-analytics-click"]').text

        address = driver.find_element_by_xpath(base_xpath + '//div[@class="secondary-attributes"]').text
        try:
            rating = driver.find_element_by_xpath(base_xpath + '//div[@class="biz-rating biz-rating-large clearfix"]/div').get_attribute('title')
            rating = float(rating.split(' ')[0])
        except:
            rating = -1
        try:
            n_reviews = driver.find_element_by_xpath(base_xpath + '//div[@class="biz-rating biz-rating-large clearfix"]/span[@class="review-count rating-qualifier"]').text
            n_reviews = int(n_reviews.split(' ')[0])
        except:
            n_reviews = 0

        restaurants.append(dict(
            city=city,
            number_restaurants=n_restaurants,
            restaurant_number=start+1,
            restaurant_name=name,
            restaurant_link=link,
            address=address,
            rating=rating,
            n_reviews=n_reviews
        ))

    driver.close()
    
restaurants = pd.DataFrame(restaurants)
restaurants.to_csv('restaurants.csv')

100%|██████████| 159/159 [1:18:37<00:00, 29.67s/it]


In [9]:
restaurants = restaurants.drop_duplicates('restaurant_link')
restaurants.to_csv('restaurants.csv')